This is the official python implementation for paper ***Neha Sharma, Chhavi Dhiman, S. Indu***, "Visual-Motion-Interaction Guided Pedestrian Intention Prediction Framework".

In [ ]:
#For importing dataset and helper codes to load and process it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/vmi'

import sys
from vmi import VMI

#Importing dataset
sys.path.insert(0,'/content/drive/MyDrive/vmi/PIE')
from pie_data import PIE

#Configuring Data options
data_opts ={'fstride': 1,
            'subset': 'default',
            'data_split_type': 'random',  # kfold, random, default
            'seq_type': 'crossing',
            'min_track_size': 75} ## for obs length of 15 frames + 60 frames tte. This should be adjusted for different setup
imdb = PIE(data_path='/content/drive/MyDrive/vmi/PIE') # change with the path to the dataset

#Configuring model options
model_opts = {'obs_input_type': ['local_box', 'local_context','pose', 'box', 'speed'],
              'enlarge_ratio': 1.5,
              'pred_target_type': ['crossing'],
              'obs_length': 15,  # Determines min track size
              'time_to_event': 60, # Determines min track size
              'dataset': 'pie',
              'normalize_boxes': True}

method_class = VMI()

#Preparing training and validation data
beh_seq_train = imdb.generate_data_trajectory_sequence('train', **data_opts)
beh_seq_val = imdb.generate_data_trajectory_sequence('val', **data_opts)

#Training and saving the model file
saved_files_path, trained_model = method_class.train(beh_seq_train, model_opts=model_opts, epochs=60, batch_size=16)


/content/drive/MyDrive/vmi
---------------------------------------------------------
Generating trajectory sequence data
fstride: 1
sample_type: all
height_rng: [0, inf]
squarify_ratio: 0
data_split_type: random
seq_type: crossing
min_track_size: 75
random_params: {'ratios': None, 'val_data': True, 'regen_data': False}
kfold_params: {'num_folds': 5, 'fold': 1}
subset: default
---------------------------------------------------------
Generating database for pie
pie annotations loaded from /content/drive/MyDrive/vmi/PIE/data_cache/pie_database.pkl
---------------------------------------------------------
Generating crossing data
Random sample currently exists.
 Loading from /content/drive/MyDrive/vmi/PIE/data_cache/random_samples.pkl
The ratios are [0.5, 0.4, 0.1]
Number of train tracks 890
Subset: train
Number of pedestrians: 890 
Total number of samples: 807 
---------------------------------------------------------
Generating trajectory sequence data
fstride: 1
sample_type: all
height

/content/drive/MyDrive/vmi/CBAM_SE.py:68: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  avg_pool = tf.compat.v1.layers.dense(inputs=avg_pool,
/content/drive/MyDrive/vmi/CBAM_SE.py:76: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  avg_pool = tf.compat.v1.layers.dense(inputs=avg_pool,
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/content/drive/MyDrive/vmi/CBAM_SE.py:87: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras

[###-----------------] 12.71% 

KeyboardInterrupt: ignored

In [ ]:
#Testing the model

saved_files_path='/content/drive/MyDrive/sf_gru/data/models/JAAD/pie/01Mar2023-05h51m06s'
beh_seq_train = imdb.generate_data_trajectory_sequence('test', **data_opts)
acc, auc, f1, precision, recall = method_class.test(beh_seq_train, saved_files_path)
